![dphi banner](https://dphi-courses.s3.ap-south-1.amazonaws.com/Datathons/dphi_banner.png)

# **Getting Started Code For [Data Sprint #44](https://dphi.tech/challenges/data-sprint-44-tweets-abuse-classification/152/overview/about) on DPhi**

# Loading Libraries
All Python capabilities are not loaded into our working environment by default (even those that are already installed in your system). So, we import each and every library that we want to use.

In data science, numpy and pandas are the most commonly used libraries. Numpy is required for calculations like mean, median, square roots, etc. Pandas is used for data processing and data frames. We choose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd).

Note: You can import all the libraries that you think will be required or can import it as you go along. 

Here we are importing two libraries - numpy and pandas


In [1]:
import numpy as np        # Fundamental package for linear algebra and multidimensional arrays
import pandas as pd       # Data analysis and manipulation tool

# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mkuma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mkuma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mkuma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Loading Dataset
Pandas module is used for reading files.

You can learn more about pandas [here](https://dphi.tech/learn/introduction-to-pandas)

In [2]:
# In read_csv() function, we have passed the location to where the files are located in the dphi official github page.
train_data = pd.read_csv("train_set_label.csv")

## What do you need to do now?
*  Perform EDA and Data Visualization, to understand the data. Learn more about EDA [here](https://dphi.tech/learn/introduction-to-exploratory-data-analysis). Learn more about data visualization [here](https://dphi.tech/learn/introduction-to-data-visualization)
*  Clean the data if required (like removing or filling missing values, treat outliers, etc.). Learn more about handling missing values [here](https://youtu.be/EaGbS7eWSs0)
*  Perform Data Preprocessing if you feel it's required. Learn one hot encoding [here](https://youtu.be/9yl6-HEY7_s).

## Basic EDA

In [3]:
train_data.head()

,tweet,label
0,start your day w your daily here,3
1,even a perfect life doesn’t feel perfect in so...,3
2,great ready for next week s q amp a with white...,3
3,5 ways to protect your mental health during th...,3
4,i m officially an occupational therapist passe...,3


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   2349 non-null   object
 1   label   2350 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 36.8+ KB


In [6]:
train_data['tweet'] = train_data['tweet'].str.lower()


def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', str(text))
    return text

train_data['tweet'] = train_data['tweet'].apply(remove_special_characters, remove_digits=False)

def f(r):
    wnl = WordNetLemmatizer()
    words = nltk.word_tokenize(r)
    lemmatized_words = [wnl.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    return " ".join(lemmatized_words)

train_data['tweet'] = train_data['tweet'].apply(f)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
data_X = cv.fit_transform(train_data['tweet']).toarray()
data_X = pd.DataFrame(data_X, columns=cv.get_feature_names())

# Separating Input Features and Output Features
Before building any machine learning model, we always separate the input variables and output variables. Input variables are those quantities whose values are changed naturally in an experiment, whereas output variable is the one whose values are dependent on the input variables. So, input variables are also known as independent variables as its values are not dependent on any other quantity, and output variable/s are also known as dependent variables as its values are dependent on other variable i.e. input variables.
By convention input variables are represented with 'X' and output variables are represented with 'y'.

In [8]:
X = data_X
y = train_data.label

# Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both input and output data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.



In [9]:
# import train_test_split
from sklearn.model_selection import train_test_split

In [10]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3, random_state = 42)

# Building Model
Now we are finally ready, and we can train the model.

There are tons of Machine Learning models like Logistic Regression, Random Forest, Decision Tree, etc. to say you some. However here we are using Logistic Regression (again, using the sklearn library).

Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [11]:
# Importing LogisticRegression from sklearn.linear_model
from sklearn.linear_model import LogisticRegression

In [12]:
lr = LogisticRegression()

### Train the model

In [13]:
lr.fit(X_train, y_train)

LogisticRegression()

# Validate The Model
Wonder🤔 how well your model learned! Lets check it.

### Predict on the validation data (X_val)
Now we predict using our trained model on the validation set we created i.e. X_val and evaluate our model on unforeseen data.

In [14]:
pred = lr.predict(X_val)

## Model Evaluation
Evaluating performance of the machine learning model that we have built is an essential part of any machine learning project. Performance of our model is done using some evaluation metrics.

There are so many evaluation metrics to use for regression problem, naming some - Accuracy Score, F1 Score, Precision, Recall etc. However, **Accuracy Score** is the metric for this challenge. 

In [15]:
# import mean squared error from sklearn.metric
from sklearn.metrics import f1_score

In [16]:
print('F1 Score is: ', f1_score(y_val, pred, average='weighted')) 

# y_val is the original target value of the validation set (X_val)
# pred is the predicted target value of the validation set

F1 Score is:  0.8906562589763701


# Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. testing_set_label.csv) given in 'Data' section of the problem page.



## Load Test Set
Load the test data on which final submission is to be made.

In [17]:
test_data = pd.read_csv('test_set_label.csv')

**Note:** 
*  Use the same techniques to deal with missing values as done with the training dataset.   

*  **Don't remove any observation/record from the test dataset otherwise you will get wrong answer. The number of items in your prediction should be same as the number of records are present in the test dataset**.

*  Use the same techniques to preprocess the data as done with training dataset.

***Why do we need to do the same procedure of filling missing values, data cleaning and data preprocessing on the new test data as it was done for the training and validation data?***

**Ans:** Because our model has been trained on certain format of data and if we don't provide the testing data of the similar format, the model will give erroneous predictions and the rmse of the model will increase. Also, if the model was build on 'n' number of features, while predicting on new test data you should always give the same number of features to the model. In this case if you provide different number of features while predicting the output, your ML model will throw a ValueError saying something like 'number of features given x; expecting n'. Not confident about these statements? Well, as a data scientist you should always perform some experiment and observe the results.



In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   784 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [19]:
test_data['tweet'] = test_data['tweet'].str.lower()


def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', str(text))
    return text

test_data['tweet'] = test_data['tweet'].apply(remove_special_characters, remove_digits=False)

def f(r):
    wnl = WordNetLemmatizer()
    words = nltk.word_tokenize(r)
    lemmatized_words = [wnl.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    return " ".join(lemmatized_words)

test_data['tweet'] = test_data['tweet'].apply(f)



In [20]:
test = cv.transform(test_data['tweet']).toarray()
test_data = pd.DataFrame(test, columns=cv.get_feature_names())

## Make Prediction on Test Dataset
Time to make submission!!!

In [21]:
target = lr.predict(test_data)

#### Note: **Follow the submission guidelines given in 'How To Submit' Section.**

## How to save prediciton results locally via jupyter notebook?
If you are working on Jupyter notebook, execute below block of codes. A file named 'prediction_results.csv' will be created in your current working directory.

In [22]:
res = pd.DataFrame(target) #target is nothing but the final predictions of your model on input features of your new unseen test data
res.columns = ["prediction"]
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR**, 
**if you are working on Google Colab then use the below set of code to save prediction results locally**

## How to save prediction results locally via colab notebook?
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(target) # target are nothing but the final predictions of your model on input features of your new unseen test data
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('submission.csv', index = False)         
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Well Done! 👍**
You are all set to make a submission. Let's head to the [challenge page](https://dphi.tech/challenges/data-sprint-44-tweets-abuse-classification/152/overview/about) to make the submission.